In [151]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import random

In [152]:
torch.cuda.is_available()

False

In [206]:
#environment class
class Environment():
    
    #initialize dataset, a random trajectory, and current cycle
    def __init__(self):
        self.dataset = pd.read_csv('train.csv')
        self.episode = self.get_trajectory(np.random.randint(low=0, high=79, size=1))
        self.cycle = 0
        
    #get random trajectory
    def get_trajectory(self, engine_id):
        return self.dataset[self.dataset.engine_id==engine_id.item()].health_indicator.to_numpy()
    
    #reset environment    
    def reset(self):
        self.cycle = 0
        self.episode = self.get_trajectory(np.random.randint(low=0, high=79, size=1))
        
    #return current state
    def get_state(self):
        return self.episode[self.cycle]
    
    #take action
    def take_action(self, action):
        if action == 0:
            if self.cycle+1 == self.episode.size:
                res = (-250, None, True)
            else:
                res = (1, self.episode[self.cycle+1], False)
        elif action == 1:
            res = (-25, self.episode[self.cycle+1], True)
        self.cycle+=1
        return res

In [218]:
class Transition():
    
    def __init__(self, state, action, state_new, reward):
        self.state = state
        self.action = action
        self.state_new = state_new
        self.reward = reward

In [219]:
#dqn model clas 
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.lin1 = nn.Linear(1,2)
        self.lin2 = nn.Linear(2,2)

    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return x

In [220]:
def get_action(net, epsilon):
    greedy = np.random.choice([True, False], p=[1-epsilon, epsilon])
    if greedy:
        action = torch.argmax(out, dim=0).item()
    else:
        action = random.choice([0,1])
    return action

In [221]:
from collections import deque

In [222]:
#hyperparameters
exp_replay_size = 128
gamma = 0.99
epsilon = 0.1
target_update_steps = 5
num_episodes = 10

#begin algorithm
QNet = DQN()
TNet = DQN()

ER = deque(maxlen = exp_replay_size)

In [226]:
def train():
    pass

In [227]:
for episode in range(num_episodes):
    
    environment = Environment()
    cummulative_reward = 0

    while True:
        
        #observation
        state = environment.get_state()
        action = get_action(state, epsilon)
        state_new, reward, terminated = environment.take_action(action)
        
        #append to replay buffer
        ER.appendleft(Transition(state, action, state_new, reward))
        
        #update variables
        cummulative_reward+=reward
        
        #train
        train()
        
        break
        